## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Описание проекта:

Цель данного проекта обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году.

Для оценки качества модли будут использоваться следущие метрики качества: RMSE, MAE и R2

# Подготовка данных

### Импрорт библиотек

In [46]:
import pandas as pd 
import numpy as np
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import avg
from pyspark.sql.functions import avg, round
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import mean
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, StringIndexer

from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

pyspark_version = pyspark.__version__
if int(pyspark_version[:1]) == 3:
    from pyspark.ml.feature import OneHotEncoder    
elif int(pyspark_version[:1]) == 2:
    from pyspark.ml.feature import OneHotEncodeEstimator
    
from pyspark.ml import Pipeline


### Инициализируем Spark сессию

In [47]:
RANDOM_SEED = 2022

spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

### Читаем содержимое файла и сохраняем в переменную

In [48]:
df = spark.read.option('header', 'true').csv('/datasets/housing.csv', inferSchema = True)

In [49]:
df.printSchema() 

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



### Выводим типы данных

In [50]:
print(pd.DataFrame(df.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


## Предобработка

### Ищем пропуски

In [51]:
for column in df.columns:
    check_col = F.col(column).isNull()
    print(column, df.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


### Заменяем пропуски на средние значения

In [52]:
# calculate mean value for column1
mean_value = df.select(mean(df["total_bedrooms"])).collect()[0][0]

# replace missing values in column1 with mean value
df = df.na.fill(mean_value, subset=["total_bedrooms"])

### Проверяем что пропуски исчезли

In [53]:
for column in df.columns:
    check_col = F.col(column).isNull()
    print(column, df.filter(check_col).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


### Создание и трансформация категориальных и некатегориальных признаков

In [54]:
stages = []

categoricalColumns = ['ocean_proximity']
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol,
                                  outputCol = categoricalCol + 'Index',
                                  handleInvalid = 'keep')
    encoder = OneHotEncoder(inputCol=stringIndexer.getOutputCol(),
                            outputCol=categoricalCol + "classVec")
    stages += [stringIndexer, encoder]

numericCols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']

assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler_all = VectorAssembler(inputCols=assemblerInputs, outputCol='features')

stages += [assembler_all]

# Обучение моделей

### Линейная регрессия (с категориальными признаками)

In [55]:
target = 'median_house_value'

train, test = df.randomSplit([.7,.3], seed=RANDOM_SEED)
print(f'Обучающая выборка содержит: {train.count()} строк, {len(train.columns)} столбцов')
print(f'Тестовая выборка содержит: {test.count()} строк, {len(test.columns)} столбцов')

Обучающая выборка содержит: 14413 строк, 10 столбцов
Тестовая выборка содержит: 6227 строк, 10 столбцов


In [56]:
lr_all = LinearRegression(labelCol='median_house_value', featuresCol='features')
stages += [lr_all]

In [57]:
pipeline_all = Pipeline(stages=stages)

In [58]:
model_all = pipeline_all.fit(train)

23/03/23 14:28:02 WARN Instrumentation: [1bea832f] regParam is zero, which might cause numerical instability and overfitting.
23/03/23 14:28:03 WARN Instrumentation: [1bea832f] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [59]:
predictions_all = model_all.transform(test)
predictedLabes_all = predictions_all.select('median_house_value', 'prediction')
predictedLabes_all.show(5) 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           85800.0|115930.10258466937|
|          103600.0|152403.52557541197|
|          106700.0|217925.05112151476|
|           50800.0| 215148.9466889538|
|           58100.0|142081.03879301064|
+------------------+------------------+
only showing top 5 rows



### Линейная регрессия без категориальных признаков ('ocean_proximity_idx', 'ocean_proximity_encoded')

In [60]:
assemblerInputs_num = numericCols
assembler_num = VectorAssembler(inputCols=assemblerInputs_num, outputCol='features')
stages[2] = assembler_num

In [61]:
pipeline_num = Pipeline(stages=stages)

In [62]:
model = pipeline_num.fit(train)

23/03/23 14:29:32 WARN Instrumentation: [6ca11a13] regParam is zero, which might cause numerical instability and overfitting.


In [63]:
predictions_num = model.transform(test)
predictedLabes_num = predictions_num.select('median_house_value', 'prediction')
predictedLabes_num.show(5) 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|           85800.0| 64956.90845325077|
|          103600.0|101431.24672467029|
|          106700.0|190825.20667221304|
|           50800.0|184188.84990955703|
|           58100.0|109444.52562058344|
+------------------+------------------+
only showing top 5 rows



Вывод метрик

In [64]:
results = []

In [65]:
def all_metrics(pred_data, model_name):
    rmse = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName='rmse').evaluate(pred_data)
    mae = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName='mae').evaluate(pred_data)
    r2 = RegressionEvaluator(predictionCol="prediction", labelCol="median_house_value", metricName='r2').evaluate(pred_data)
    return model_name, rmse, mae, r2

In [66]:
results.append(all_metrics(predictedLabes_all,'features'))
results.append(all_metrics(predictedLabes_num, 'numerical_features'))

In [67]:
results = pd.DataFrame(results, columns=['Model', 'RMSE', 'MAE', 'R2'])
results.style.highlight_min(axis = 0)

,Model,RMSE,MAE,R2
0,features,68408.293044,49963.174372,0.648230
1,numerical_features,69222.260031,50933.703413,0.639809


In [68]:
spark.stop()

# Анализ результатов

### Выводы

Линейная регрессия без категориальных значений показала в среднем лучшие метрики чем с категориальными значениями:

С категориальными значениями:
Root Mean Squared Error (RMSE) = 68408.293044
Mean Absolute Error (MAE) = 49963.174372
R2 Score = 0.648230


Без категориальных значений:
Root Mean Squared Error (RMSE) = 69222.260031
Mean Absolute Error (MAE) = 50933.703413
R2 Score = 0.639809

Мы видим, что RMSE и MAE выше в модели где кат. значения были убраны и не принимали участиче в обучении модели.